In [1]:
%pip install -r requirements.txt
%pip install pyserial 
%pip install keyboard

  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for tk==0.1.0 from https://files.pythonhosted.org/packages/1e/0b/029cbdb868bb555fed99bf6540fff072d500b3f895873709f25084e85e33/tk-0.1.0-py3-none-any.whl.metadata
  Using cached tk-0.1.0-py3-none-any.whl.metadata (693 bytes)
Using cached tk-0.1.0-py3-none-any.whl (3.9 kB)
  Running setup.py clean for pyaudio
Failed to build pyaudio
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [9 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      copying src\pyaudio.py -> build\lib.win-amd64-cpython-311
      running build_ext
      building '_portaudio' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
ERROR: Could not build wheels for pyaudio, which is required to install pyproject.toml-based projects


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Module 1 - Connecting with and Controlling KITT

Let us start with providing an overview and a brief description on getting to know the vehicle and implementing some basic control scripts. 

## Brief Overview

KITT is a remotely controlled vehicle. To allow for this remote control, a wireless link must be used.
Over this wireless link, control commands can be sent from the base station (your computer) to KITT.
This wireless link also facilitates requests for data from KITT and KITT’s data in response to these
requests.
This project requires you to maneuver KITT in all sorts of ways. In this module, you will set up a
connection to KITT and work with basic controls to allow for more advanced actions later on.
This module starts with an assignment to get to know your team, set up your development environment,
and start working with the car.

**Learning Objectives:**
- Getting to know your team
- How to set up a Bluetooth connection to KITT
- How to send commands to KITT
- Write a script to drive KITT using keyboard commands

**Deliverable**
- In the Brightspace assignment folder: submit a filled-out document epo4_kickoff.docx de-
scribing how you want to collaborate as a team.
- Basic scripts for driving KITT around using keyboard commands.

**Preparation**
- Read this module
- Look at the described KITT commands.
- Make sure Python IDE is ready.

**Time Duration**
One-and-a-half lab session and one preparation session at home.

**What is needed**

- KITT, with fresh batteries
- Laptop/PC running Python
- KITT datasheets, found on Brightspace
- epo4_kickoff.docx, found on Brightspace

## KITT Basics

KITT is based on a repurposed toy car model, the Traxass E-MAXX. As seen in this {ref}`KITT car <car-figure>` figure. There is a data sheet (here it is {doc}`C:\Users\aparn\mybookname\EE2L21 KITT datasheet.pdf`) as well which gives an overview of the various components, such as the motor controller, LCD status indicator, ultrasonic sensors, and, most importantly, the communication module.


```{figure} traxxas_e-maxx.png
---
height/width: 150px
name: car-figure
---
Traxass E-MAXX along with it's dimensions
```

### Hardware

Many components are combined on a single custom-made MCU board installed on KITT. The microcontroller forms the core of this board, an NXP LPC4357 chip with an ARM CORTEX-M4/M0 core. The MCU board contains the Bluetooth module, connectors for all peripherals, and an amplifier for the audio beacon, which will be used for the localization. The following figure {ref}`Schematic Overview of KITT communication with the PC <Schematic-overview>` shows a schematic overview of the connection.

The MCU is fed from the rechargeable batteries using a buck converter that converts the voltage to *5V* DC. Using an LM1117-3.3, *3.3V* is generated.


```{figure} communication_overview.png
---
height/width: 150px
name: Schematic-overview
---
Schematic of communication with KITT and PC.
```

The Bluetooth communication with KITT consists of the following elements:

-  On car: Roving Networks RN-41 I/RM, Onboard Bluetooth module with UART control.
-  On PC: LM Technologies LM506, USB Bluetooth V4.0 dongle with Broadcom BCM20702 chipset, or the internal Bluetooth module of your laptop.

You can use your computer's internal Bluetooth connector to connect to KITT. If you want to use one of the lab computers or have trouble connecting to KITT from your PC (check if all drivers are installed), you can use the USB Bluetooth dongle. 

KITT has four LED's, see figure {ref}`KITT <KITT>`, their meaning is as follows:

```{figure} figurejoin.png
---
height/width: 80px
name: KITT
---
(KITT)
```

- Red (twice): *5V* and *20V* supply voltages are present.
- Green (blinking): Bluetooth searches for connection; (steady) Bluetooth connected.
- Yellow (blinking): Bluetooth data transfer.
 
The button next to the LEDs is a reset button for the MCU. This reset button allows the user to reset KITT's actions if it freezes or is unresponsive.

## Communicating with KITT

For this module, you will write several scripts fundamental for working with KITT for the rest of the project. For this, we will use the Python module pySerial to send and receive data to/from KITT

### pySerial

pySerial is a Python module that provides a simple and efficient way to communicate with serial ports. It allows Python programs to access and manipulate the serial ports on a computer, allowing them to communicate with other devices connected to those ports. With pySerial, you can easily send and receive data to/from these devices. pySerial is cross-platform, which works on Windows, macOS, and Linux operating systems.

- *serial.Serial(port, baud rate)* - This command initializes a serial connection. The port argument specifies the serial port to use (e.g., "COM1" on Windows or "/dev/rfcomm0" on Linux),
and the baud-rate argument specifies the data rate in bits per second.
- *serial.write(data)* - This command sends data over the serial connection. The data argument is a bytes object that contains the data to be sent.
- *serial.read(size)* - This command reads a specified number of bytes from the serial connection. It blocks until the specified number of bytes is received.
- *serial.flush()* - This command is used to flush the input and output buffers of the serial connection.
- *serial.close()* - This command is used to close the serial connection.

These are just a few of the basic commands used in pySerial. Depending on the specific use case, many more commands and options are available. Further more, updated information regarding the module can also be found through this link [pySerial](https://pyserial.readthedocs.io/en/latest/pyserial.html).

### Connecting to KITT 

Before you can do anything with KITT, you must be able to connect to it. In this section we provide some basic commands to do this.

To start off with, connect the car to your computer over Bluetooth. Make sure you know the name of the Bluetooth port that the car is connected to. To access the link in Python use, 

```{code-cell}

serial_port = serial.Serial(comport, 115200, rtscts=True)
# If your transmission happens over port 10
comport = 'COM10'
```

```{important} Click to Show

serial_port is an instance of pySerial and this object can be used to manipulate KITT. The variable comport is the path to the comport assigned to the Bluetooth connection with KITT. Which
comport specifically KITT is connected to can be found in your communication settings.
Further information regarding the issues with connecting to the port via Bluetooth can be solved by referring to the Appendices.

Also keep in mind, The Bluetooth connection is disturbed by leaving comports open, quitting your work without closing the communication link, removing the Bluetooth dongle, or turning off the Bluetooth connection. Avoid disturbing the Bluetooth connection by ending the connection properly before doing any of the aforementioned things. If you disturb the Bluetooth link, you may need to reboot your computer to reset the operating system, which costs you valuable lab time.

```
The connection must also be closed once you are done using the port, 

```{code-cell}
serial_port.close()
```

Connecting to a serial port over Bluetooth can be pretty tricky. Windows users, refer to Appendix B. Linux users can use '# rfcomm bind rfcomm0 MAC_ADDRESS'; there is also a section about it on the Arch [Linux wiki](https://wiki.archlinux.org/title/Bluetooth#Bluetooth_serial). The MAC address of KITT always ends with the code printed on its label. For example, if the label reads DA:84, the MAC will end with xx:xx:xx:xx:DA:84.
To find out which serial ports are available, you can use the list_ports tool of pySerial. Run the
following in a shell: 

python -m serial.tools.list_ports



### Controlling KITT

Now that you can connect to KITT, you will write commands to control KITT. First off, for *driving* there are 2 types of instructions:

-  A direction command.
-  A motor speed command.

These are controlled using Pulse width modulation.  Both commands are neutral at a setting of 150. The direction commands range from 200 (hard left) to 100 (hard right), and the motor commands range from 135 (backward) to 165 (forward).


```{note} Click to Show

However, there is a dead zone for the motor commands, so KITT will likely not start moving forward until the PWM is set to about 153. It is recommended to experiment with these values, they are also battery-dependent. You should test the size of the drive command dead-band, verify that 150 is the middle position for steering (sometimes there is a deviation of ±2), and test the maximum left and right position.

```
All the commands are sent in binary and end with a new line character.



```{code-cell}

serial_port.write(b'code\n')

```

To set direction to 169 and motor speed to 135, it can be done as follows, 

```{code-cell}
serial_port.write(b'D169\n')
serial_port.write(b'M135\n')
```

```{attention}

Once you set the motor speed commands, KITT will continue to act on this until you either
transmit a new motor speed command or reset the MCU using the button on KITT, which will set both
direction and motor speed commands to the neutral value of 150. Do one of these two options after your
tests since KITT will keep driving if you don’t.

```

Secondly, *Audio beacon* commands used are,

```{code-cell}
# ON-OFF commands
serial_port.write(b'A1\n')
serial_port.write(b'A0\n')
```

```{caution}

Be aware that the default code word for the beacon is 0x00000000, which means KITT will not start making noise on its own when the beacon is turned on. You should specify a code as described below before you can hear the beacon make noise. The beacon signal is similar to what was used in EE2T11 Telecommunication A practicum, except that now it is possible to use an arbitrary carrier frequency, bit
frequency, and repetition count. Furthermore, an arbitrary carrier frequency, bit frequency and repitition count can be used (Generally, maximum of 30 kHz).

```
```{code-cell}
# Carrier frequency set to 10000 Hz for example.

carrier_frequency = 10000.to_bytes(2, byteorder='big')
serial_port.write(b'F' + carrier_frequency + b'\n')

# Furthermore, you can set the bit-frequency of a code that is transmitted with OOK on the carrier frequency to, for example, 5000 Hz by using

bit_frequency = 5000.to_bytes(2, byteorder='big')
serial_port.write(b'B' + bit_frequency + b'\n')

# The repetition count, which sets the time between consecutive transmission and has a minimum of 32, can be set to 2500 for example,

repetition_count = 2500.to_bytes(2, byteorder='big')
serial_port.write(b'R' + repetition_count + b'\n')

# The 32 bits code pattern is transmitted bit-wise over the beacon. Hence, code must be specified in hexadecimal. Say you use the hexadecimal 0xDEADBEEF as an example; the command to do this is,

code = 0xDEADBEEF.to_bytes(4, byteorder='big')
serial_port.write(b'C' + code + b'\n')
```
The repitition count formula is as follows, 

$$
\displaystyle
\left( Repitition count = Bit frequency/ Repitition Frequency \right)
$$

Lastly, the *status command* which requests for the status report and all the data from KITT by sending a request, 

```{code-cell}

serial_port.write(b'S\n')
status = serial_port.read_until(b'\x04')

```

```{Note}

The status string reports the current drive commands, the ultrasonic sensor distance (cm), the battery voltage (mV), the audio status (on/off) and control parameters (code word,carrier-frequency, bit-frequency, repetition count). A sensor distance of 999 means overload (i.e., out of range) 
Also, remember that all the numbers provided here serve only as an example. It is up to you to determine what carrier frequency, code word, etc., best fits the goal of succeeding at the final challenge.

```

## Implementing a basic KITT Control Script

This assignment is part of the communication mid-term assessment. You are asked to write a Python
program to control your car connected over Bluetooth. The program should allow the car to be controlled
using the keyboard w, a, s, and d keys. The e and q should start and stop the audio beacon. Also add a
key to stop communication with KITT.
The template script below defines a class KITT with essential methods for communication, and your
task is to complete the script by implementing the wasd function

In [2]:

import serial
import keyboard 
import numpy as np
import matplotlib.pyplot as plt

In [9]:


class KITT:
    def __init__(self, port, baudrate=115200):
        self.serial = serial.Serial(port, baudrate, rtscts=True)
        
    def send_command(self, command):
        self.serial.write(command.encode())

    def set_speed(self, speed):
        self.send_command(f'M{speed}\n')

    def set_angle(self, angle):
        self.send_command(f'D{angle}\n')

    def stop(self):
        self.set_speed(150)
        self.set_angle(150)

    def __del__(self):
        self.serial.close()



In [10]:
def wasd(kitt):
    # add your code

    if __name__ == "__main__":
        kitt_instance = KITT("your_port_here")
        wasd(kitt_instance)

Below is a short explanation of the code. Ensure you understand how the code works before using its
various functions.
**if __name__ == "__main__"**
- This is the code that will start executing when you run your script. This contstruct allows you to
import your script as a module into another script without running the test code.
- An instance of KITT (an object of the KITT class) is then created, with the port as a parameter.
You should change this to your port number.
- Next, the wasd function is called and given this instance of KITT. It should detect key-press events
and attach these to KITT commands like
– kitt.set_speed(170)
– kitt.set_angle(150)
– kitt.sto

**KITT Class**

- The **__init__** method is the one that runs when you make an instance of the class. It starts the
serial communication with the specified port and baud rate.
- send_command method sends a given command to KITT over the established serial connection.
- set_speed and set_angle methods adjust KITT’s speed and steering angle, respectively.
- stop method brings KITT to a halt by setting both speed and angle to a neutral state. 
- **__del__** method ensures the proper closure of the serial communication when the KITT object is
deleted. It runs automatically at the end of the script.
- **TODO:** modify the **__init__** so that when the communication with KITT is started, the beacon
is initialized with the correct set of parameters. Use the existing serial port and send_command.
- **TODO**: add two methods start_beacon and stop_beacon that turn the beacon on or off. Note
that you should have set the beacon parameters during the **__init__**, so there is no need to resend
them every time you turn the beacon on



**TODO: wasd Function**

- The wasd function is designed to be a continuous loop that reads keyboard events using the keyboard library which you loaded using import keyboard.
- When a key is pressed (KEY_DOWN), the function interprets the key and adjusts KITT’s speed and
steering angle or toggles the beacon accordingly.
- The 'w' key accelerates KITT forward, 's' stops KITT, ’a’ turns KITT left, and 'd' turns KITT
right.
- The 'e' key turns on the beacon, and the 'q' key turns off the beacon.
- When a key is released (KEY_UP), you could define appropriate actions, e.g. stop KITT or reset the
steering angle.


#### Optional Extensions

If you would like to go above and beyond with the task then below listed are a few tips which can help with extending the functionality of the program,

- Add a GUI: You could create a graphical user interface (GUI) for the program to make it more user-friendly. The GUI could display information about the car, such as its speed or battery level,
and provide buttons for controlling the car. You can use the built-in tkinter module but you are free to use any other GUI module, for example PyGame, pyglet, or PyQt.
- Add error handling: Currently, if there is an error with the Bluetooth connection or the serial communication, the program will simply crash. You could add some error handling to handle
these cases more gracefully, such as printing an error message and exiting the program.
- Add speed control: Currently, the program only supports moving the car forward or backward and turning left or right. You could add support for controlling the speed of the car, such as by
sending different commands to the car depending on how long the user holds down the forward or backward keys.
- Add an emergency stop: KITT doesn’t have a brake; the 150 speed setting lets KITT roll out to standstill, which might take a long distance. Add an emergency brake by letting KITT drive
backwards for a short period of time. (You should first detect if the previous speed setting caused KITT to move forward. You need to define a state variable to memorize the speed setting.)

#### Midterm Assessment Report

After you finish this assignment, and ultimo in week 4, showcase the functionality of your script to your
assigned TA. After you pass this assessment, you are ready to document your results in your midterm
report.
For this Module, you would include a chapter that covers the approach, implementation, testing and
results of the basic controller; but postpone the documentation of a GUI to the final report. Review
Chapter 7 for guidelines.
Remember to document your code, using comments to define input/output variables of functions and to
explain the logic and any modifications made. Your completed script will be crucial for the upcoming
challenges, contributing to the overall autonomous driving system.
After you have completed this module and have tested all the components thoroughly, start on the second
part of the communication script outlined in Module 2.

## Appendix: Dive deeper into Fundamental Concepts 

### PWM (Pulse Width Modulation)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def generate_pwm_waveform(duty_cycle, frequency=1, t_end=1, samples=1000):
    """
    Generates a PWM signal.
    Parameters:
        duty_cycle (float): The duty cycle of the PWM signal (percentage of time signal is high).
        frequency (float): The frequency of the PWM signal in Hz.
        t_end (float): The end time for the generated signal.
        samples (int): Number of samples in the generated signal.
    Returns:
        t (numpy array): Time vector.
        pwm_signal (numpy array): The PWM signal.
    """
    t = np.linspace(0, t_end, samples)
    period = 1 / frequency
    on_time = period * (duty_cycle / 100)
    pwm_signal = (t % period < on_time).astype(float)
    return t, pwm_signal

In [ ]:
frequency = 1  # Hz
duty_cycles = [10, 30, 50, 70, 90]  # Different duty cycles

plt.figure(figsize=(10, 8))

In [ ]:
# Generate and plot PWM for different duty cycles
for i, duty_cycle in enumerate(duty_cycles, 1):
    t, pwm_signal = generate_pwm_waveform(duty_cycle, frequency)
    plt.subplot(len(duty_cycles), 1, i)
    plt.plot(t, pwm_signal, drawstyle='steps-post')
    plt.ylim(-0.2, 1.2)
    plt.title(f'PWM Signal - Duty Cycle: {duty_cycle}%')
    plt.ylabel('Amplitude')
    plt.grid(True)

plt.xlabel('Time (seconds)')
plt.tight_layout()
plt.show()

### On-Off Keying

It is a simple form of amplitude-shift keying (ASK), which is a modulation technique used in digital communication. OOK represents digital data as the presence or absence of a carrier wave. In its simplest form, "1" is represented by the presence of the carrier (an "on" state), and "0" is represented by its absence (an "off" state).

To illustrate OOK using Python and matplotlib, I'll generate a simple OOK signal based on a digital binary input string. This example will help visualize how OOK transmits digital data.

In [ ]:
def generate_ook_signal(data, bit_duration=1, sampling_rate=100):
    """
    Generates an OOK signal from binary data.
    Parameters:
        data (str): Binary string to be transmitted.
        bit_duration (float): Duration of each bit in seconds.
        sampling_rate (int): Number of samples per second.
    Returns:
        t (numpy array): Time vector for the signal.
        ook_signal (numpy array): The OOK signal.
    """
    # Number of samples per bit
    samples_per_bit = int(bit_duration * sampling_rate)
    
    # Generate the signal
    ook_signal = np.repeat([int(bit) for bit in data], samples_per_bit)
    total_samples = len(ook_signal)
    t = np.linspace(0, total_samples / sampling_rate, total_samples, endpoint=False)
    
    return t, ook_signal

# Binary data to be transmitted
data = "1100101011110001"

# Generate OOK signal
t, ook_signal = generate_ook_signal(data)

# Plotting
plt.figure(figsize=(12, 3))
plt.step(t, ook_signal, where='post')
plt.ylim(-0.2, 1.2)
plt.title('OOK Signal Representation')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()
